In [2]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [5]:
#Take a SC
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
55 documents
9 classes ['angry', 'frustated', 'funny', 'goodbye', 'greeting', 'sad', 'songs-happy', 'songs-sad', 'thanks']
54 unique stemmed words ['!', '#', '$', "'m", "'s", 'a', 'aghhh', 'alon', 'am', 'angry', 'anyon', 'ar', 'bad', 'bye', 'can', 'day', 'do', 'feel', 'for', 'funny', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'jok', 'know', 'lat', 'lot', 'me', 'mind', 'not', 'on', 'play', 'sad', 'see', 'so', 'someth', 'song', 'tel', 'thank', 'that', 'ther', 'u', 'ur', 'what', 'whts', 'wtygs', 'yo', 'you']


In [6]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [7]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
tf.compat.v1.reset_default_graph()


Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [8]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [9]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [10]:
#TAake as SC
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')


Training Step: 6999  | total loss: 0.38472 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.38472 - acc: 0.7322 -- iter: 48/55
Training Step: 7000  | total loss: 0.39692 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.39692 - acc: 0.7322 -- iter: 55/55
--
Saving the Model.......
INFO:tensorflow:C:\Users\joysh\Desktop\Minor Project\AI_ChatBot_Python\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\joysh\Desktop\Minor Project\AI_ChatBot_Python\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\joysh\Desktop\Minor Project\AI_ChatBot_Python\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\joysh\Desktop\Minor Project\AI_ChatBot_Python\model.tflearn.meta
INFO:tensorflow:100


In [11]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [12]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from C:\Users\joysh\Desktop\Minor Project\AI_ChatBot_Python\model.tflearn


In [13]:
#TAke a SC
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [15]:
#Final Loop SC
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- hi
Hi there, what can I do for you?
You- i am sad
Perhaps You would like to listen to music
You- yes
Please listen to: <a href='https://www.youtube.com/watch?v=4NRXx6U8ABQ'>Blinding lights</a>
You- thaks
...
You- thanks
My pleasure
You- i feel better now
Oh! I am Sorry, what can I do to cheer you up?
You- thanks
Any time!
You- bye
See you later, thanks for visiting.Enter quit to exit


KeyboardInterrupt: Interrupted by user

In [16]:
pip install pyaudio


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\joysh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
#Final Loop SC
while True:
    try:
        print('speak')
        with sr.Microphone() as source2:


            r.adjust_for_ambient_noise(source2, duration=0.2)

            audio2 = r.listen(source2)

            MyText = r.recognize_google(audio2)
            MyText = MyText.lower()

            print("MyText")
    #         SpeakText(MyText)
            input_data = MyText
            answer = response(input_data)
            print('BOT:',answer)
    except:
        input_data = input("You- ")
        answer = response(input_data)
        answer

speak
Did you say hello
Hey :-)
BOT: None
speak
Did you say can you play a song
Perhaps You can ask me for a joke?
BOT: None
speak
